Selecting optimal confusion sets where the POS doesn't change according to context



In [ ]:
# drive
from google.colab import drive
drive.mount('/content/drive')

#pos tagging
import spacy

# standard library
import numpy as np
import pandas as pd
import tensorflow as tf

#removes punctuation for confusion set
import string

#LLM
!pip install --q transformers
from transformers import RobertaTokenizer, TFRobertaModel, pipeline, TFRobertaForMaskedLM

# progress bar
from tqdm import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load spacy for English: English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
nlp = spacy.load("en_core_web_sm")

#note: there is also a transformer model, trained on RoBERTa base as well
# but evaluation metrics for POS tagging is similar in both
#nlp = spacy.load("en_core_web_trf")



In [ ]:
def check_pos(word, sentence, pos_tag):
    doc = nlp(sentence)
    word_pos = None

    for token in doc:
        if token.text == word:
            word_pos = token.pos_
            break

    if word_pos == pos_tag:
      output = f"POS tag' {word_pos}' is as expected "

    else:
      output = f"The word' {word}' plays another role: {word_pos} "




    return output

Random thought that lead nowhere 1:


Just though of how to use them:

I think the sensible use of a confusion set is to have a small number of words that are frequently confused among themselves. This is the case for determiners, such as [a, an, the]. I can imagine a scenario where the autotranscription is 'a' but it the label is 'an'. So what we could do is:

Find POS in sentence.
If word in autotranscription (or it's phonetic equivalent) is in our  mini_confusion_set, then create k-beams, for k in confusion set.
Else, leave as is.

So for example:
Label: "He ate an apple"
Autotranscription: "E ate eh apehl"
POS: ['NOUN', 'VERB', 'INTJ', 'NOUN']

(Didn't work, since eh is tagged as INTJ)

In [ ]:
sentence = "E ate eh apehl"

makes_sense = [token.pos_ for token in nlp(sentence)]
makes_sense

['NOUN', 'VERB', 'INTJ', 'NOUN']

In [ ]:
#nonsense sentence, just for funsies
nonsense = "zinting fromplins bamphed ostrically ovzer fle flope besfohret dellluging zae snergs miskh migglesnaps slurtwring enk gloaglming."
whaaaattt = [token.pos_ for token in nlp(nonsense)]
whaaaattt

['VERB',
 'NOUN',
 'VERB',
 'ADV',
 'ADJ',
 'PROPN',
 'NOUN',
 'VERB',
 'VERB',
 'ADJ',
 'NOUN',
 'ADJ',
 'NOUN',
 'VERB',
 'NOUN',
 'VERB',
 'PUNCT']

# AUXILIARY VERBS

In [ ]:
old_aux = ['am', 'is', 'are', 'was', 'were', 'being', 'been', 'be',
      'will', 'would','shall', 'should', 'may', 'might', 'must', 'can', 'could']

# after checking for options where the words are ambiguous
new_aux = ['am', 'are', 'is', 'was', 'were','shall', 'should',  'being', 'been', 'be',
'would', 'could']

In [ ]:
aux_pos = [[token.pos_ for token in nlp(sentence)]
    for sentence in aux
]

aux_pos

[['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX'],
 ['AUX']]

## am, are

In [ ]:
# am is good to go
print(check_pos("am", "I am", 'AUX'))
print(check_pos("am", "I am able to go", 'AUX'))

print(check_pos("are", "You are", 'AUX'))
print(check_pos("are", "They are going", 'AUX'))

POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## is, was

In [ ]:
# so is was (and where, in our logic)
print(check_pos("was", "I was going", 'AUX'))
print(check_pos("was", "I was there", 'AUX'))


POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
The word' where' plays another role: SCONJ 
The word' where' plays another role: SCONJ 


## were

In [ ]:

print(check_pos("were", "where were you going", 'AUX'))
print(check_pos("were", "were tou able to feed it ?", 'AUX'))

POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## shall, should

In [ ]:
# shall looks good too
print(check_pos("shall", "this time next week I shall be in Scotland", 'AUX'))
print(check_pos("shall", "they shall succeed", 'AUX'))
print(check_pos("shall", "every employer shall take all practicable steps to ensure the safety of employees", 'AUX'))
print(check_pos("shall", "shall I send you the book?", 'AUX'))

print(check_pos("should", "this time next week I should be in Scotland", 'AUX'))
print(check_pos("should", "they should succeed", 'AUX'))
print(check_pos("should", "every employer should take all practicable steps to ensure the safety of employees", 'AUX'))
print(check_pos("should", "should I send you the book?", 'AUX'))

POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## will - exclude

In [ ]:
# will can sometimes be a noun
print(check_pos("will", "He signed his will", 'AUX'))
print(check_pos("will", "I will be crying by tomorrow", 'AUX'))
print(check_pos("will", "will I get the same results ?", 'AUX'))

The word' will' plays another role: NOUN 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## might - exclude?? In the first example it plays another role, but recognized as aux

In [ ]:
print(check_pos("might", "With great might he struck the sword", 'AUX'))
print(check_pos("might", "I might meet you tomorrow", 'AUX'))
print(check_pos("might", "might you be interested in ice cream ?", 'AUX'))

POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## must - exclude?? in the first example, plays role of noun, but not recognized

In [ ]:
# must can sometimes be a noun, but spacy doesn't think so
print(check_pos("must", "The smell of must", 'AUX'))
print(check_pos("must", "must it be so ?", 'AUX'))

POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## being - exclude

In [ ]:
#being can sometimes be a NOUN
print(check_pos("being", "She is a human being", 'AUX'))
print(check_pos("being", "You are being obnoxious", 'AUX'))
print(check_pos("being", "They were being held prisoners", 'AUX'))

The word' being' plays another role: NOUN 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## been

In [ ]:
print(check_pos("been", "They had been to the show", 'AUX'))


POS tag' AUX' is as expected 


## be

In [ ]:
print(check_pos("be", "Let him be", 'AUX'))
print(check_pos("be", "The symbol of Berylium is be", 'AUX'))
print(check_pos("be", "be that as it may", 'AUX'))

POS tag' AUX' is as expected 
POS tag' AUX' is as expected 
POS tag' AUX' is as expected 


## can - exclude

In [ ]:
#can can sometimes be a noun
expect_verb = "Can you bring me a can of Coke ? "

expect_verb_pos = [token.pos_ for token in nlp(expect_verb)]
print(expect_verb,expect_verb_pos[0], expect_verb_pos[5])



Can you bring me a can of Coke ?  AUX NOUN


## may - exclude

In [ ]:
#may can sometimes be a noun
expect_verb = "May May get a cat in May ? "

expect_verb_pos = [token.pos_ for token in nlp(expect_verb)]
print(expect_verb,expect_verb_pos[0], expect_verb_pos[1], expect_verb_pos[6])

May May get a cat in May ?  AUX AUX PROPN


# ADPOSITIONS

In [ ]:
old_adp = ["about", "at", "by", "for", "from", "in", "of", "on", "to", "with",
        "into", "during", "including", "until", "against", "among",
        "throughout", "despite", "towards", "upon", "concerning"]

#remove words in our old list not labelled as ADP
new_adp = ['at','by','for','from','of','on','with','into','during','against','among','throughout',
                 'toward']

In [ ]:
adp_pos = [[{token.text:token.pos_} for token in nlp(sentence)]
    for sentence in old_adp
]

print(adp_pos)



[[{'about': 'ADV'}], [{'at': 'ADP'}], [{'by': 'ADP'}], [{'for': 'ADP'}], [{'from': 'ADP'}], [{'in': 'ADP'}], [{'of': 'ADP'}], [{'on': 'ADP'}], [{'to': 'PART'}], [{'with': 'ADP'}], [{'into': 'ADP'}], [{'during': 'ADP'}], [{'including': 'VERB'}], [{'until': 'ADP'}], [{'against': 'ADP'}], [{'among': 'ADP'}], [{'throughout': 'ADP'}], [{'despite': 'SCONJ'}], [{'towards': 'ADP'}], [{'upon': 'SCONJ'}], [{'concerning': 'VERB'}]]


## at

In [ ]:
print(check_pos("at", "staying at a hotel", 'ADP'))
print(check_pos("at", "the symbol for atmosphere is at", 'ADP'))
print(check_pos("at", "my instagram handle is at catlover", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## by

In [ ]:
print(check_pos("by", "I want a house by the beach", 'ADP'))
print(check_pos("by", "saw him go by", 'ADP'))
print(check_pos("by", "by the way", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## for

In [ ]:
print(check_pos("for", "for what it's worth, I liked it", 'ADP'))
print(check_pos("for", "what is it for", 'ADP'))
print(check_pos("for", "acted for the best", 'ADP'))
print(check_pos("for", "all for the plan", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## from

In [ ]:
print(check_pos("from", "came here from the city", 'ADP'))
print(check_pos("from", "cost from $5 to $10", 'ADP'))


POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## in - exclude

In [ ]:
print(check_pos("in", "It was sunny in the morning but became cloudy in the afternoon .", 'ADP'))
print(check_pos("in", "I went in its stead .", 'ADP'))
print(check_pos("in", "Did you go in ?", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
The word' in' plays another role: ADV 


## of

In [ ]:
print(check_pos("of", "of the people", 'ADP'))
print(check_pos("of", "The review of the book was quite critical.", 'ADP'))



POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## on

In [ ]:
print(check_pos("on", "let's get it on", 'ADP'))
print(check_pos("on", "it's on the counter", 'ADP'))
print(check_pos("on", "I'm on it", 'ADP'))
print(check_pos("on", "She kept calm and carried on", 'ADP'))
print(check_pos("on", "They live on a pension", 'ADP'))
print(check_pos("on", "Right on cue", 'ADP'))
print(check_pos("on", "on the up and up", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## with

In [ ]:
print(check_pos("with", "had a fight with his brother", 'ADP'))
print(check_pos("with", "off with his head", 'ADP'))
print(check_pos("with", "a dress identical with her hostess's", 'ADP'))


POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## into

In [ ]:
print(check_pos("into", "came into the house", 'ADP'))
print(check_pos("into", "dividing 3 into 6 gives 2", 'ADP'))
print(check_pos("into", "not into her music", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## during

In [ ]:
print(check_pos("during", "swims every day during the summer", 'ADP'))
print(check_pos("during", "during the interview, they asked about my previous jobs .", 'ADP'))



POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## until - exclude

In [ ]:
print(check_pos("until", "stayed until morning", 'ADP'))
print(check_pos("until", "not available until tomorrow", 'ADP'))
print(check_pos("until", "never able to relax until he took up fishing", 'ADP'))
print(check_pos("until", "ran until she was breathless", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
The word' until' plays another role: SCONJ 
The word' until' plays another role: SCONJ 


## against

In [ ]:
print(check_pos("against", "spoke against his enemies", 'ADP'))
print(check_pos("against", "against the law", 'ADP'))
print(check_pos("against", "she sat down just over against me", 'ADP'))
print(check_pos("against", "profits are up against last year", 'ADP'))
print(check_pos("against", "throw on another log of wood against father comes home.", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## among

In [ ]:
print(check_pos("among", "hidden among the trees", 'ADP'))
print(check_pos("among", "made a fortune among themselves", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## throughout

In [ ]:
print(check_pos("throughout", "cities throughout the United States", 'ADP'))
print(check_pos("throughout", "remained loyal throughout", 'ADP'))
print(check_pos("throughout", "troubled her throughout her life", 'ADP'))

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 


## toward

In [ ]:
print(check_pos("toward", "driving toward town", 'ADP'))
print(check_pos("toward", "an attitude toward life", 'ADP'))
print(check_pos("toward", "proceeds go toward the establishment of a scholarship", 'ADP'))
print(check_pos("toward", "a toward breeze", 'ADP')) #this is obsolete, so we're good

POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
POS tag' ADP' is as expected 
The word' toward' plays another role: ADJ 


# DET

In [ ]:
old_det = ['the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'your', 'his',
       'her', 'its', 'our', 'their', 'all', 'both', 'half', 'either', 'neither',
       'each', 'every', 'other', 'another', 'such', 'what', 'rather', 'quite']

# only words with
cand_det = ['the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'your', 'his',
       'her', 'its', 'our', 'their', 'all', 'both', 'half', 'either', 'neither',
       'each', 'every', 'other', 'another', 'such', 'what', 'rather', 'quite']

new_det = ['the', 'a', 'an', 'this', 'that', 'these', 'those', 'my']

In [ ]:
print(check_pos("some", "some manner of hardship'", 'DET'))
print(check_pos("alll", "alll manner of hardship'", 'DET'))
print(check_pos("all", "all manner of hardship'", 'DET'))



POS tag' DET' is as expected 
The word' alll' plays another role: NOUN 
POS tag' DET' is as expected 


## the

In [ ]:
print(check_pos("the", "put the cat out", 'DET'))
print(check_pos("the", "none the wiser for attending", 'DET'))
print(check_pos("the", "a dollar the dozen", 'DET'))


POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 


## a

In [ ]:
print(check_pos("a", "a man overboard", 'DET'))
print(check_pos("a", "a cat", 'DET'))
print(check_pos("a", "twice a week", 'DET'))
print(check_pos("a", "There are a lot a stairs, but the view at the top is absolutely beautiful.", 'DET'))


POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 


## an

In [ ]:
print(check_pos("an", "an opera", 'DET'))
print(check_pos("an", "sixty miles an hour", 'DET'))



POS tag' DET' is as expected 
POS tag' DET' is as expected 


## this - remove

In [ ]:
print(check_pos("this", "I can only say this : it wasn't here yesterday", 'DET'))
print(check_pos("this", "this book is mine", 'DET'))
print(check_pos("this", "then this guy runs in", 'DET'))
print(check_pos("this", "didn't expect to wait this long", 'DET'))




The word' this' plays another role: PRON 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
The word' this' plays another role: PRON 


## that - remove

In [ ]:
print(check_pos("that", "that is my father", 'DET'))
print(check_pos("that", "after that I went to bed", 'DET'))
print(check_pos("that", "the purest water is that produced by distillation", 'DET'))
print(check_pos("that", "that's a cat — quick and agile", 'DET'))
print(check_pos("that", "he was helpful, and that to an unusual degree", 'DET'))
print(check_pos("that", "said that he was afraid", 'DET'))
print(check_pos("that", "it is unlikely that he'll be in", 'DET'))
print(check_pos("that", "that rarity among leaders", 'DET'))
print(check_pos("that", "the house that Jack built", 'DET'))
print(check_pos("that", "that which", 'DET'))
print(check_pos("that", "a nail about that long", 'DET'))
print(check_pos("that", "did not take the festival that seriously", 'DET'))




The word' that' plays another role: PRON 
The word' that' plays another role: PRON 
The word' that' plays another role: PRON 
The word' that' plays another role: PRON 
The word' that' plays another role: SCONJ 
The word' that' plays another role: SCONJ 
The word' that' plays another role: SCONJ 
POS tag' DET' is as expected 
The word' that' plays another role: PRON 
POS tag' DET' is as expected 
The word' that' plays another role: PRON 
The word' that' plays another role: PRON 


## these

In [ ]:
print(check_pos("these", "I can only say these words : it wasn't here yesterday", 'DET'))
print(check_pos("these", "these books are mine", 'DET'))
print(check_pos("these", "then these guys run in", 'DET'))
print(check_pos("these", "didn't expect to wait these many minutes", 'DET'))

POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 


## those

In [ ]:
print(check_pos("those", "I can only say those words : it wasn't here yesterday", 'DET'))
print(check_pos("those", "those books are mine", 'DET'))
print(check_pos("those", "then those guys run in", 'DET'))
print(check_pos("those", "didn't expect to wait those many minutes", 'DET'))

POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 


## my - exclude

In [ ]:
print(check_pos("my", "my car", 'DET'))
print(check_pos("my", "my favorite TV show was on last night .", 'DET'))
print(check_pos("my", "But for me, a great pastry has the ability to brighten my day, impress a group of friends or make someone feel loved .", 'DET'))

The word' my' plays another role: PRON 
The word' my' plays another role: PRON 
The word' my' plays another role: PRON 


## your - remove

In [ ]:
print(check_pos("your", "your bodies", 'DET'))
print(check_pos("your", "when you face the north, east is at your right", 'DET'))
print(check_pos("your", "Please wash your hands before dinner .", 'DET'))
print(check_pos("your", "Don't worry about me — you've got your own problems .", 'DET'))



The word' your' plays another role: PRON 
The word' your' plays another role: PRON 
The word' your' plays another role: PRON 
The word' your' plays another role: PRON 


## his - remove

In [ ]:
print(check_pos("his", "his house", 'DET'))
print(check_pos("his", "He sat quietly at his desk.", 'DET'))
print(check_pos("his", "He sat quietly at his desk.", 'DET'))



The word' his' plays another role: PRON 
The word' his' plays another role: PRON 
The word' his' plays another role: PRON 


## its - exclude

In [ ]:
print(check_pos("its", "going to its kennel", 'DET'))
print(check_pos("its", "a suite with its own private bathroom", 'DET'))
print(check_pos("its", "its final enactment into law", 'DET'))
print(check_pos("its", "Let the medicine do its job .", 'DET'))
print(check_pos("its", "The door shuts on its own .", 'DET'))



The word' its' plays another role: PRON 
The word' its' plays another role: PRON 
The word' its' plays another role: PRON 
The word' its' plays another role: PRON 
The word' its' plays another role: PRON 


## our - exclude

In [ ]:
print(check_pos("our", "going to our kennel", 'DET'))
print(check_pos("our", "a suite with our own private bathroom", 'DET'))
print(check_pos("our", "our final enactment into law", 'DET'))
print(check_pos("our", "Let us do our job .", 'DET'))
print(check_pos("our", "We shut the door on our own .", 'DET'))

The word' our' plays another role: PRON 
The word' our' plays another role: PRON 
The word' our' plays another role: PRON 
The word' our' plays another role: PRON 
The word' our' plays another role: PRON 


## their - exclude

In [ ]:
print(check_pos("their", "going to their kennel", 'DET'))
print(check_pos("their", "a suite with their own private bathroom", 'DET'))
print(check_pos("their", "their final enactment into law", 'DET'))
print(check_pos("their", "Let them do their job .", 'DET'))
print(check_pos("their", "They shut the door on their own .", 'DET'))

The word' their' plays another role: PRON 
The word' their' plays another role: PRON 
The word' their' plays another role: PRON 
The word' their' plays another role: PRON 
The word' their' plays another role: PRON 


## all - exclude

In [ ]:
print(check_pos("all", "needed all the courage they had", 'DET'))
print(check_pos("all", "all manner of hardship", 'DET'))
print(check_pos("all", "spoke in all seriousness", 'DET'))
print(check_pos("all", "all five children were present", 'DET'))
print(check_pos("all", "all the angles of a triangle are equal to two right angles", 'DET'))
print(check_pos("all", "beyond all doubt", 'DET'))
print(check_pos("all", "became all attention", 'DET'))
print(check_pos("all", "all ears", 'DET'))
print(check_pos("all", "sat all alone", 'DET'))
print(check_pos("all", "all-league halfback", 'DET'))
print(check_pos("all", "all that I have", 'DET'))
print(check_pos("all", "gave equal attention to all", 'DET'))
print(check_pos("all", "gave his all for the cause", 'DET'))



POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
The word' all' plays another role: PRON 
POS tag' DET' is as expected 
The word' all' plays another role: ADV 
POS tag' DET' is as expected 
The word' all' plays another role: PRON 
The word' all' plays another role: PRON 
The word' all' plays another role: PRON 


## both - exclude

In [ ]:
print(check_pos("both", "both of us", 'DET'))
print(check_pos("both", "prized both for its beauty and for its utility", 'DET'))
print(check_pos("both", "both these armies", 'DET'))


The word' both' plays another role: PRON 
The word' both' plays another role: CCONJ 
The word' both' plays another role: CCONJ 


## half - exclude

In [ ]:
print(check_pos("half", "half the distance", 'DET'))
print(check_pos("half", "half past two", 'DET'))
print(check_pos("half", "the score was tied at the half", 'DET'))
print(check_pos("half", "a half share", 'DET'))
print(check_pos("half", "half measures", 'DET'))
print(check_pos("half", "her singing isn't half bad", 'DET'))
print(check_pos("half", "half persuaded", 'DET'))
print(check_pos("half", "the crowd was half jeering , half respectful", 'DET'))
print(check_pos("half", "too clever by half", 'DET'))
print(check_pos("half", "half again as many", 'DET'))
print(check_pos("half", "cut the apple in half", 'DET'))



POS tag' DET' is as expected 
The word' half' plays another role: ADV 
The word' half' plays another role: NOUN 
The word' half' plays another role: ADJ 
The word' half' plays another role: NOUN 
The word' half' plays another role: ADV 
The word' half' plays another role: NOUN 
The word' half' plays another role: ADV 
The word' half' plays another role: NOUN 
The word' half' plays another role: ADV 
The word' half' plays another role: NOUN 


## either - exclude

In [ ]:
print(check_pos("either", "flowers blooming on either side of the walk", 'DET'))
print(check_pos("either", "take either road", 'DET'))
print(check_pos("either", "take either of the two routes", 'DET'))
print(check_pos("either", "can be used either as a guest room or as an office", 'DET'))
print(check_pos("either", "not smart or handsome either", 'DET'))
print(check_pos("either", "who answers for the Irish parliament ? or army either ?", 'DET'))



POS tag' DET' is as expected 
The word' either' plays another role: CCONJ 
The word' either' plays another role: PRON 
The word' either' plays another role: CCONJ 
The word' either' plays another role: ADV 
The word' either' plays another role: ADV 


## neither - exclude

In [ ]:
print(check_pos("neither", "neither black nor white", 'DET'))
print(check_pos("neither", "neither did I", 'DET'))
print(check_pos("neither", "rigid enforcement of antique decorum will help neither language , literature , nor literati", 'DET'))
print(check_pos("neither", "neither of those ideal solutions are in sight", 'DET'))
print(check_pos("neither", "neither hand", 'DET'))
print(check_pos("neither", "just as the serf was not permitted to leave the land, so neither was his offspring", 'DET'))

The word' neither' plays another role: CCONJ 
The word' neither' plays another role: CCONJ 
The word' neither' plays another role: CCONJ 
The word' neither' plays another role: CCONJ 
The word' neither' plays another role: CCONJ 
The word' neither' plays another role: CCONJ 


## each - exclude

In [ ]:
print(check_pos("each", "to each his own", 'DET'))
print(check_pos("each", "cost a dollar each", 'DET'))
print(check_pos("each", "A rope was tied to each end of the boat .", 'DET'))
print(check_pos("each", "We were allowed two tries each .", 'DET'))


POS tag' DET' is as expected 
The word' each' plays another role: PRON 
POS tag' DET' is as expected 
The word' each' plays another role: PRON 


## every - exclude

In [ ]:
print(check_pos("every", "every once in a while", 'DET'))
print(check_pos("every", "was given every chance", 'DET'))
print(check_pos("every", "we have every confidence in her", 'DET'))


The word' every' plays another role: PRON 
POS tag' DET' is as expected 
POS tag' DET' is as expected 


##other - exclude

In [ ]:
print(check_pos("other", "held on with one hand and waved with the other one", 'DET'))
print(check_pos("other", "taller than the other boys", 'DET'))
print(check_pos("other", "every other day", 'DET'))
print(check_pos("other", "any other color would have been better", 'DET'))
print(check_pos("other", "sold in the U . S . and 14 other countries", 'DET'))
print(check_pos("other", "the other evening", 'DET'))
print(check_pos("other", "in other times", 'DET'))
print(check_pos("other", "went from one side to the other", 'DET'))
print(check_pos("other", "something or other", 'DET'))
print(check_pos("other", "was unable to see them other than by going to their home", 'DET'))



The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 
The word' other' plays another role: ADJ 


## another - exclude

In [ ]:
print(check_pos("another", "the same scene viewed from another angle", 'DET'))
print(check_pos("another", "do it another time", 'DET'))
print(check_pos("another", "have another piece of pie", 'DET'))
print(check_pos("another", "one copy to send out , another for the files", 'DET'))
print(check_pos("another", "Complaining about problems is one thing , but finding solutions to them is another.", 'DET'))
print(check_pos("another", "in one way or another", 'DET'))



POS tag' DET' is as expected 
POS tag' DET' is as expected 
POS tag' DET' is as expected 
The word' another' plays another role: PRON 
The word' another' plays another role: PRON 
The word' another' plays another role: PRON 


## such - exclude

In [ ]:
print(check_pos("such", "a bag such as a doctor carries", 'DET'))
print(check_pos("such", "his excitement was such that he shouted", 'DET'))
print(check_pos("such", "in the past few years many such women have shifted to full - time jobs", 'DET'))
print(check_pos("such", "never heard such a hubbub", 'DET'))
print(check_pos("such", "other such clinics throughout the state", 'DET'))
print(check_pos("such", "such was the result", 'DET'))
print(check_pos("such", "tin and glass and such", 'DET'))
print(check_pos("such", "such tall building", 'DET'))
print(check_pos("such", "hasn't been in such good spirits lately", 'DET'))
print(check_pos("such", "related such that each excludes the other", 'DET'))


The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 
POS tag' DET' is as expected 
The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 
The word' such' plays another role: ADJ 


## what - exclude

In [ ]:
print(check_pos("what", "what is this ?", 'DET'))
print(check_pos("what", "You said what ?", 'DET'))
print(check_pos("what", "what do you think I am, a fool ?", 'DET'))
print(check_pos("what", "what , no breakfast?", 'DET'))
print(check_pos("what", "you know what", 'DET'))
print(check_pos("what", "A clever play, what ?", 'DET'))
print(check_pos("what", "what angered us was the tone of the article.", 'DET'))


The word' what' plays another role: PRON 
The word' what' plays another role: PRON 
The word' what' plays another role: PRON 
The word' what' plays another role: PRON 
The word' what' plays another role: PRON 
The word' what' plays another role: PRON 
The word' what' plays another role: PRON 


## rather - exclude

In [ ]:
print(check_pos("rather", "this you should pity rather than despise", 'DET'))
print(check_pos("rather", "would rather read than watch television", 'DET'))
print(check_pos("rather", "my father , or rather my stepfather", 'DET'))
print(check_pos("rather", "was no better but rather grew worse", 'DET'))
print(check_pos("rather", "spent rather a lot of money", 'DET'))
print(check_pos("rather", "it ' s rather warm", 'DET'))


The word' rather' plays another role: ADV 
The word' rather' plays another role: ADV 
The word' rather' plays another role: ADV 
The word' rather' plays another role: ADV 
The word' rather' plays another role: ADV 
The word' rather' plays another role: ADV 


## quite - exclude

In [ ]:
print(check_pos("quite", "not quite finished", 'DET'))
print(check_pos("quite", "quite a beauty", 'DET'))
print(check_pos("quite", "quite near", 'DET'))


The word' quite' plays another role: ADV 
POS tag' DET' is as expected 
The word' quite' plays another role: ADV 
